In [ ]:
# Directory manipulation for relative notebook imports
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

# Relative import for relevant methods
from Visualization.plot_util import *
from Visualization.projection_plots import *
from Data.data_load_util import *
from Simulation.projections_util import *
from Models.Neat.neat_util import *
from Simulation.projections import *


# Imports for ipynb visualization
import plotly.io as pio



In [ ]:
# Data loading (including projections)
zips_df, state_df, pos_df = make_dataset(granularity='both', remove_outliers=False, load_dir_prefix='../Data/')

#update zips_df with 2025 panels
projection2025 = get_baseline_2025(zips_df, state_df, save="../Data/Baseline_2025/baseline.pkl", load="../Data/Baseline_2025/baseline.pkl")
zips_df = updated_df_with_picks(zips_df, projection2025.panel_placements)

#process data for EVA
data_manager = DataManager(zips_df, fields=['Median_income', 'carbon_offset_kg_per_panel', 'yearly_sunlight_kwh_kw_threshold_avg', 'panel_utilization', 'black_prop'])

proj_path_2m = "../Simulation/Projection_Saves/projections_2m_from_2025.pkl"
projections_2m = create_projections(zips_df, state_df, n_panels=2000000, save=proj_path_2m, load=proj_path_2m)
# projections = create_projections(zips_df, state_df, n_panels=2000000, save="../Simulation/Projection_Saves/projections_2mil.pkl", load="../Simulation/Projection_Saves/projections_2mil.pkl")

In [ ]:
#get the EVA projection
with open(dir1 + "\\Models\\Neat\\models\\05-30-25\\NEAT_model_lexicase_weighted_3.pkl", 'rb') as f:
    network = pickle.load(f)

#2 million panels
neat_proj_path_2m = "../Simulation/Projection_Saves/projections_neat_2m_from_2025.pkl"
neat_proj_new_2m = create_neat_proj(data_manager, 2000000, NeatModel(network), create_paper_objectives(), save=neat_proj_path_2m, load=neat_proj_path_2m)


In [ ]:
# State map of carbon offset per panel added
plot_state_map(state_df, key='carbon_offset_metric_tons_per_panel', fill_color='Blues', legend_name="Carbon Offset Metric Tons Per Panel", save_dir_prefix="../Visualization/")


In [ ]:
pio.renderers.default = "notebook"
geo_plot(zips_df['carbon_offset_metric_tons_per_panel'], 'mint', "Carbon Offset Per Panel", pos_df)

In [ ]:
#fig 3
methods_colorpalette = ["#4a2377","#8cc5e3","#0d7d87", "#f55f74"]

compared_projections_2m = [projections_2m[1], projections_2m[2], projections_2m[3]] + [neat_proj_new_2m]
plot_projections(compared_projections_2m, "Carbon Offset", color_palette=methods_colorpalette, interval=100000)
plot_projections(compared_projections_2m, "Energy Potential", color_palette=methods_colorpalette, interval=100000)

In [ ]:
#fig 4
coverage_ratio = 674914/1600000 #ratio of panels covered by our dataset
annual_new_panels = [coverage_ratio * 720000 * 1.09**i for i in range(11)]

#get the baseline
installations2025 = 1973556
baseline2025 = {key: projection2025.objective_projections[key][installations2025] for key in projection2025.objective_projections.keys()}

for num_panels in [sum(annual_new_panels[:i]) for i in [1,2,5]]:
    plot_bar_comparison_baseline_ratio(baseline2025, [projections_2m[2], projections_2m[3], neat_proj_new_2m], panel_count=num_panels, color_palette = methods_colorpalette, hatches=["\\","/","x"], hatch_size=0.5, fontsize=22)


In [ ]:
#get absolute projected objectives from EVA after 5 years (2030) and 10 years (2035)
for i in [5,10]:
    num_panels = sum(annual_new_panels[:i])
    proj1 = projections_2m[1].interpolateObjectiveProjections(num_panels)
    proj2 = neat_proj_new_2m.interpolateObjectiveProjections(num_panels)
    print("Years:", i)
    print("SQ Carbon offset:",proj1['Carbon Offset'])
    print("EVA Carbon offset:",proj2['Carbon Offset'])
    print("Carbon ratio:", proj2['Carbon Offset']/proj1['Carbon Offset'])

    print("SQ Energy Potential:",proj1['Energy Potential'])
    print("EVA Energy Potential:",proj2['Energy Potential'])
    print("Energy ratio:", proj2['Energy Potential']/proj1['Energy Potential'])
    print("----")